In [2]:
from src import parse_miami
from src import cm_metrics
from src import calc_cocoa as cc
from src import new_metric_strat as nms
from collections import defaultdict
# from src import calc_cm_strategies as ccs

mi_utt_file = './data/all_cm/miami_all_ids.0211'  # miami_all.0211

data_folder_path = "./data/miami/clean_1208/"
miami_data = parse_miami.load_data(data_folder_path)

In [55]:
print miami_data['zeledon8']['FLA']['words'][:3]
print miami_data['zeledon8']['FLA']['uttid'][:3]
print miami_data['zeledon8']['FLA']['words_01'][:3]

[['uhuh_eng'], [], ['ya_spa', 'estamos_spa', 'dentro_spa', 'que_spa']]
['mi_zeledon8_0002_FLA', 'mi_zeledon8_0009_FLA', 'mi_zeledon8_0010_FLA']
[[1], [], [0, 0, 0, 0]]


In [ ]:
reload(cm_metrics)
txt_dict, strat_dict = cm_metrics.get_style_metrics_miami(miami_data)
# # print miami_dict.keys()  # ['neither', 's-0', 's-1', 'c-1', 'c-0']
# print len(strat_dict.keys())  # 2889

In [128]:
# print strat_dict.keys()[0] # mi_zeledon7_0809_JAS
spkr_csid = defaultdict(list)
for csid in strat_dict:
    spkr_csid[csid.split('_')[3]].append(csid)
print len(spkr_csid) # 80
print spkr_csid.keys()
print 'ELI' in spkr_csid

80
['BRA', 'EVN', 'JES', 'LIL', 'DIE', 'TOM', 'EMI', 'EVE', 'MIG', 'CON', 'TIM', 'NIC', 'PAI', 'NAT', 'AAR', 'FLA', 'LUI', 'ISA', 'LUK', 'ETH', 'PAO', 'CAL', 'ARI', 'JAD', 'HER', 'SAM', 'SOF', 'OLI', 'ROB', 'AME', 'JUL', 'JOC', 'ELL', 'SEB', 'SEA', 'CAR', 'SAR', 'ASH', 'GAB', 'GRA', 'CHA', 'ELE', 'IAN', 'VAN', 'AUD', 'VAL', 'VIC', 'ABI', 'FEL', 'CHL', 'JAS', 'CLA', 'IRI', 'JAM', 'MEG', 'GIA', 'LAU', 'LAN', 'HEN', 'GIL', 'JEN', 'JAC', 'REB', 'LEA', 'MAS', 'MAR', 'MAT', 'MAY', 'MEL', 'ABE', 'AVE', 'NOA', 'RIC', 'ANT', 'MAD', 'ALY', 'KEV', 'KAY', 'AVA', 'LAR']
False


In [168]:
import csv
socio_map = defaultdict(dict)
lang_spkr_map = {}
# ex: socio_map['IRI'] = {'PR': 'PR', 'sp': 4, 'en': 4}
nat_map = defaultdict(list)
lang_map = defaultdict(list)
with open('data/miami/miami_spkrs_socio_121019.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        spkr = row['spkr']
        if spkr not in spkr_csid:
            print 'not CS', spkr
        socio_map[spkr]['nation'] = row['nation']
        spa_abil = int(row['spa_abil'])
        en_abil = int(row['en_abil'])
        socio_map[spkr]['sp'] = spa_abil
        socio_map[spkr]['en'] = en_abil
        nat_map[row['nation']].append(spkr)
        
        if spa_abil == en_abil:
            lang_key = 'sym'
        elif spa_abil > en_abil:
            lang_key = 'dom_spa'
        else:
            lang_key = 'dom_eng'
        lang_map[lang_key].append(spkr)
        lang_spkr_map[spkr] = lang_key

not CS COL
not CS ELI


In [149]:
print {key: len(val) for key, val in lang_map.iteritems()}

{'dom_spa': 8, 'sym': 61, 'dom_eng': 14}


In [106]:
# get nat of spkrs >= 3
nat_hifreq = {}
for nat in sorted(nat_map.keys()):
    if len(nat_map[nat]) > 2:
        nat_hifreq[nat] = nat_map[nat]

In [169]:
for nat, spkrs in nat_hifreq.iteritems():
    print nat, len(spkrs), Counter([lang_spkr_map[spkr] for spkr in spkrs])
print sum([len(spkrs) for spkrs in nat_hifreq.values()])

PR 3 Counter({'dom_spa': 1, 'dom_eng': 1, 'sym': 1})
Dominican 3 Counter({'sym': 2, 'dom_eng': 1})
Cuba 18 Counter({'sym': 12, 'dom_spa': 4, 'dom_eng': 2})
Venezuela 3 Counter({'sym': 3})
US 23 Counter({'sym': 15, 'dom_eng': 7, 'dom_spa': 1})
5 5 Counter({'sym': 4, 'dom_spa': 1})
Colombia 7 Counter({'sym': 7})
US/Cuba 8 Counter({'sym': 7, 'dom_eng': 1})
Nicaragua 4 Counter({'sym': 3, 'dom_spa': 1})
Ecuador 3 Counter({'sym': 3})
77


In [133]:
# from collections import Counter
strat_ratio = {}
coarse_strat_ratio = {}
spkr_numutts = {}
for nat, spkrs in nat_hifreq.iteritems():
    for spkr in spkrs:
        if spkr not in spkr_csid:
            print 'SPKR not present', spkr
            continue
        csids = spkr_csid[spkr]
        ct_strat = Counter([strat_dict[csid] for csid in csids])
        coarse_ct_strat = {'n': ct_strat['neither'], 's': ct_strat['s-1'] + ct_strat['s-0'], 'c': ct_strat['c-1'] + ct_strat['c-0']}
#         print spkr, coarse_ct_strat
        sum_utt = sum(ct_strat.values())
        strat_ratio[spkr] = {key: float(val)/sum_utt for key, val in ct_strat.iteritems()}
        coarse_strat_ratio[spkr] = {key: float(val)/sum_utt for key, val in coarse_ct_strat.iteritems()}
        spkr_numutts[spkr] = sum_utt

SPKR not present ELI


In [134]:
print strat_ratio['IRI']
print spkr_numutts['MAR']

{'neither': 0.3488372093023256, 's-1': 0.16279069767441862, 'c-1': 0.023255813953488372, 'c-0': 0.46511627906976744}
513


In [159]:
# calc avg % strategies (fine-grain)
# 2 categories: US & Other
# import statistics as stat
other_spkr_ratios = defaultdict(list)
for nat, spkrs in nat_hifreq.iteritems():
    if nat == 'US':
        continue
    
    for spkr in spkrs:
        if spkr not in strat_ratio:
            continue
        for strat, val in strat_ratio[spkr].iteritems():
            other_spkr_ratios[strat].append(val)
            
us_spkr_ratios = defaultdict(list)
for spkr in nat_hifreq['US']:
    if spkr not in strat_ratio:
        continue
    for strat, val in strat_ratio[spkr].iteritems():
        us_spkr_ratios[strat].append(val)
            
for strat_tot, strat_lst in other_spkr_ratios.iteritems():
    print '{}\t{}\t{}'.format('Other', strat_tot, stat.mean(strat_lst))
for strat_tot, strat_lst in us_spkr_ratios.iteritems():
    print '{}\t{}\t{}'.format('American', strat_tot, stat.mean(strat_lst))

Other	neither	0.381448112603
Other	s-0	0.180071690849
Other	c-1	0.198733003294
Other	s-1	0.170800426008
Other	c-0	0.307145557827
American	neither	0.453949528272
American	c-0	0.265978473064
American	c-1	0.153505336926
American	s-1	0.144071347516
American	s-0	0.139064663057


In [136]:
# calc avg % strategies (coarse-grain)
# 2 categories: US & Other
# import statistics as stat
other_spkr_ratios = defaultdict(list)
for nat, spkrs in nat_hifreq.iteritems():
    if nat == 'US':
        continue
    
    for spkr in spkrs:
        if spkr not in spkr_csid:
            continue
        for strat, val in coarse_strat_ratio[spkr].iteritems():
            other_spkr_ratios[strat].append(val)
            
us_spkr_ratios = defaultdict(list)
for spkr in nat_hifreq['US']:
    if spkr not in spkr_csid:
        continue
    for strat, val in coarse_strat_ratio[spkr].iteritems():
        us_spkr_ratios[strat].append(val)
            
for strat_tot, strat_lst in other_spkr_ratios.iteritems():
    print '{}\t{}\t{}'.format('Other', strat_tot, stat.mean(strat_lst))
for strat_tot, strat_lst in us_spkr_ratios.iteritems():
    print '{}\t{}\t{}'.format('American', strat_tot, stat.mean(strat_lst))

Other	s	0.26605952336
Other	c	0.380747779785
Other	n	0.353192696854
American	s	0.225335614875
American	c	0.341348926319
American	n	0.433315458805


In [167]:
# calc avg % strategies (coarse-grain)
# all non-US categories
other_spkr_ratios = defaultdict(list)
for nat, spkrs in nat_hifreq.iteritems():
#     if nat == 'US':
#         continue
    
    for spkr in spkrs:
        if spkr not in spkr_csid:
            continue
        for strat, val in coarse_strat_ratio[spkr].iteritems():
            other_spkr_ratios[strat].append(val)
    for strat_tot in ['s', 'c', 'n']:
        print '{}\t{}\t{}'.format(nat, strat_tot, stat.mean(other_spkr_ratios[strat_tot]))

PR	s	0.188522825151
PR	c	0.508161068045
PR	n	0.303316106804
Dominican	s	0.221154949146
Dominican	c	0.449915344373
Dominican	n	0.32892970648
Cuba	s	0.253267863585
Cuba	c	0.387893241524
Cuba	n	0.358838894891
Venezuela	s	0.245213113713
Venezuela	c	0.396704393433
Venezuela	n	0.358082492854
US	s	0.236288522398
US	c	0.371850918402
US	n	0.3918605592
5	s	0.242391934787
5	c	0.367493944181
5	n	0.390114121032
Colombia	s	0.253998413978
Colombia	c	0.35697272357
Colombia	n	0.389028862452
US/Cuba	s	0.257919654604
US/Cuba	c	0.363265007395
US/Cuba	n	0.378815338001
Nicaragua	s	0.251576487927
Nicaragua	c	0.367883309434
Nicaragua	n	0.380540202639
Ecuador	s	0.254271023536
Ecuador	c	0.369342848519
Ecuador	n	0.376386127945


In [ ]:
# get distribution of num utts by nationality
# print spkr_numutts
for nat, spkrs in nat_hifreq.iteritems():
    if nat != 'US':
        continue
#     print spkrs
    for spkr in spkrs:
        if spkr not in spkr_csid:
            continue
        print spkr_numutts[spkr]

In [170]:
# calc lang strat (fine)
lang_spkr_ratios = {}
for lang_prof, spkrs in lang_map.iteritems():
    lang_spkr_ratios[lang_prof] = defaultdict(list)
    for spkr in spkrs:
        if spkr not in strat_ratio:
            continue
            
        for strat, val in strat_ratio[spkr].iteritems():
            lang_spkr_ratios[lang_prof][strat].append(val)

for lang_key, lang_strats in lang_spkr_ratios.iteritems():
    for strat_tot, strat_lst in lang_strats.iteritems():
        print '{}\t{}\t{}'.format(lang_key, strat_tot, stat.mean(strat_lst))


dom_spa	neither	0.398085016835
dom_spa	s-0	0.128914141414
dom_spa	c-1	0.22037037037
dom_spa	s-1	0.144848484848
dom_spa	c-0	0.41632996633
sym	neither	0.401319497691
sym	c-0	0.308176572059
sym	c-1	0.187952815418
sym	s-1	0.165585206443
sym	s-0	0.172304852518
dom_eng	neither	0.412780933913
dom_eng	s-0	0.163443484371
dom_eng	s-1	0.160642588824
dom_eng	c-1	0.167157367973
dom_eng	c-0	0.147859446779


In [160]:
# calc lang strat (fine)
lang_spkr_ratios = {}
for lang_prof, spkrs in lang_map.iteritems():
    lang_spkr_ratios[lang_prof] = defaultdict(list)
    for spkr in spkrs:
        if spkr not in strat_ratio:
            continue
            
        for strat, val in coarse_strat_ratio[spkr].iteritems():
            lang_spkr_ratios[lang_prof][strat].append(val)

for lang_key, lang_strats in lang_spkr_ratios.iteritems():
    for strat_tot, strat_lst in lang_strats.iteritems():
        print '{}\t{}\t{}'.format(lang_key, strat_tot, stat.mean(strat_lst))


dom_spa	s	0.154987373737
dom_spa	c	0.446927609428
dom_spa	n	0.398085016835
sym	s	0.259331301593
sym	c	0.375181298724
sym	n	0.365487399683
dom_eng	s	0.296845492466
dom_eng	c	0.290373573622
dom_eng	n	0.412780933913


In [ ]:
# get distribution of num utts by lang prof
# print spkr_numutts
for lang_prof, spkrs in lang_map.iteritems():
    print lang_prof
    for spkr in spkrs:
        if spkr not in strat_ratio:
            continue
        print spkr_numutts[spkr]